# Circuit Decomposition 
Edge and gate decomposition with zero classical communication.

Prerequisites: 

1. Create a new Conda environment
```shell
conda create --name azure-knitting python=3.9.17
conda activate azure-knitting
```

1. Install everything needed to run the `modified-circuit-knitting-toolbox` locally: 
```shell
cd modified-circuit-knitting-toolbox
python -m pip install -e .
python -m pip install 'docplex>=2.23.222' 'cplex>=22.1.0.0' 'gurobipy==11.0.0'
```

1. Install everything needed to run on Azure:
```shell
python -m pip install --upgrade 'azure-quantum[qiskit]==1.2.1'
# python -m pip install qiskit-aer==0.12.1
```

### Documentation
Running on Azure: [instructions](https://learn.microsoft.com/en-us/azure/quantum/quickstart-microsoft-qiskit?tabs=tabid-quantinuum&pivots=platform-local#prerequisites)


First, we enable communication with our Azure instance

In [13]:
# Prevent from overriding already run experiments
# exit()

In [14]:
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider


workspace = Workspace(
            resource_id = "/subscriptions/4ed1f7fd-7d9e-4c61-9fac-521649937e65/resourceGroups/Cutting/providers/Microsoft.Quantum/Workspaces/Cutting",
            location = "eastus")


provider = AzureQuantumProvider(workspace)

Check what Azure targets are available. Note that Quantinuum may need to be enabled in your workspace.

In [15]:
print("This workspace's targets:")
for backend in provider.backends():
    print("- " + backend.name())

This workspace's targets:
- quantinuum.sim.h1-1sc
- quantinuum.sim.h1-1sc
- quantinuum.sim.h1-1e
- quantinuum.sim.h1-1e
- quantinuum.qpu.h1-1
- quantinuum.qpu.h1-1


In [16]:
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.circuit.library import EfficientSU2

# Create a quantum circuit to cut. We create a simple ansatz
mapper = JordanWignerMapper()
ansatz = EfficientSU2(3, reps=1)
# Decompose to the actual individual gates
circuit = ansatz.decompose(reps=3)
# Set some arbitrary parameters
circuit.assign_parameters([0.8] * len(circuit.parameters), inplace=True)

print(circuit)

global phase: 3.8832
     ┌─────────────┐┌─────────────┐                    ┌─────────────┐»
q_0: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├────────────■───────┤ U3(0.8,0,0) ├»
     ├─────────────┤├─────────────┤          ┌─┴─┐     ├─────────────┤»
q_1: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├──■───────┤ X ├─────┤ U3(0.8,0,0) ├»
     ├─────────────┤├─────────────┤┌─┴─┐┌────┴───┴────┐├─────────────┤»
q_2: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ X ├┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├»
     └─────────────┘└─────────────┘└───┘└─────────────┘└─────────────┘»
«     ┌─────────────┐
«q_0: ┤ U3(0,0,0.8) ├
«     ├─────────────┤
«q_1: ┤ U3(0,0,0.8) ├
«     └─────────────┘
«q_2: ───────────────
«                    


Create a list of observables to use, and automatically find cut locations.

In [17]:
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import cut_wires_and_gates_to_subcircuits

observables = ["ZZI", "IZZ", "IIZ", "XIX", "ZIZ", "IXI"]

subcircuits, subobservables = cut_wires_and_gates_to_subcircuits(
    circuit=circuit,
    observables=observables,
    method='automatic',
    max_subcircuit_width=2,
    max_cuts=9,
    num_subcircuits=[2]
)

Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 22.1.1.0 | 2023-06-15 | d64d5bd77
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 2 times.
MIP Presolve eliminated 19 rows and 7 columns.
MIP Presolve modified 3 coefficients.
Aggregator did 24 substitutions.
Reduced MIP has 11 rows, 8 columns, and 35 nonzeros.
Reduced MIP has 5 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.12 ticks)
Found incumbent of value 2.000000 after 0.01 sec. (0.13 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 11 rows, 8 columns, and 35 nonzeros.
Reduced MIP has 5 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 1.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic

In [18]:
# Visualize the subcircuits. Note the decomposed 2-qubit gates marked 'cut_cx_0'
for key in subcircuits.keys():
    print(subcircuits[key])

      ┌─────────────┐┌─────────────┐                 ┌─────────────┐»
q9_0: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├──────────────■──┤ U3(0.8,0,0) ├»
      ├─────────────┤├─────────────┤┌──────────┐┌─┴─┐├─────────────┤»
q9_1: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ cut_cx_0 ├┤ X ├┤ U3(0.8,0,0) ├»
      └─────────────┘└─────────────┘└──────────┘└───┘└─────────────┘»
«      ┌─────────────┐
«q9_0: ┤ U3(0,0,0.8) ├
«      ├─────────────┤
«q9_1: ┤ U3(0,0,0.8) ├
«      └─────────────┘
     ┌─────────────┐┌─────────────┐┌──────────┐┌─────────────┐┌─────────────┐
q10: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ cut_cx_0 ├┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├
     └─────────────┘└─────────────┘└──────────┘└─────────────┘└─────────────┘


In [19]:
from circuit_knitting.cutting.gate_and_wire_cutting.evaluation import azure_queue_experiments


# Use the Quantinuum emulator backend
# FIXME: use syntax checker for early testing
backend_str = 'quantinuum.sim.h1-1e'
backend = provider.get_backend(backend_str)

# Submit the subcircuits to Azure Quantum
job_list, qpd_list, coefficients, subexperiments = azure_queue_experiments(
    circuits=subcircuits,
    subobservables=subobservables,
    num_samples=8,  # 8 unique samples to get some statistics
    backend = backend,
    # provider = provider,
    shots=128  # Balance of cost and accuracy
)

Check on the actual results from run on Quantinuum.

In [20]:
from circuit_knitting.cutting.gate_and_wire_cutting.evaluation import get_experiment_results_from_jobs

experiment_results = get_experiment_results_from_jobs(job_list, qpd_list, coefficients)
experiment_results

..............................................

CuttingExperimentResults(quasi_dists=[[[({2: 0.28125, 0: 0.34375, 3: 0.2890625, 1: 0.0859375}, 0), ({2: 0.0859375, 0: 0.5234375, 3: 0.0390625, 1: 0.3515625}, 0)], [({0: 0.1015625, 1: 0.8984375}, 0), ({0: 0.2578125, 1: 0.7421875}, 0)]], [[({2: 0.0234375, 0: 0.7890625, 3: 0.15625, 1: 0.03125}, 0), ({2: 0.6015625, 0: 0.09375, 3: 0.09375, 1: 0.2109375}, 0)], [({0: 0.34375, 1: 0.65625}, 0), ({0: 1.0}, 0)]], [[({5: 0.1328125, 2: 0.0625, 4: 0.0703125, 6: 0.171875, 1: 0.0078125, 7: 0.0234375, 0: 0.53125}, 1), ({5: 0.046875, 6: 0.03125, 2: 0.265625, 4: 0.2734375, 1: 0.0390625, 3: 0.0078125, 7: 0.078125, 0: 0.2578125}, 1)], [({0: 0.5703125, 1: 0.4296875}, 0), ({0: 0.6484375, 1: 0.3515625}, 0)]], [[({5: 0.125, 4: 0.015625, 2: 0.0703125, 6: 0.1640625, 7: 0.015625, 0: 0.609375}, 1), ({5: 0.0234375, 4: 0.25, 2: 0.296875, 6: 0.015625, 1: 0.015625, 7: 0.0625, 0: 0.3359375}, 1)], [({0: 0.1171875, 1: 0.8828125}, 0), ({0: 0.640625, 1: 0.359375}, 0)]], [[({2: 0.2265625, 0: 0.484375, 3: 0.2890625}, 0), ({2

Now check the actual expectation values from the observables.

In [21]:
from circuit_knitting.cutting.cutting_reconstruction import reconstruct_expectation_values

quantinuum_expvals = reconstruct_expectation_values(*experiment_results, subobservables)
quantinuum_expvals

[0.320556640625,
 0.514404296875,
 0.3489990234375,
 0.1357421875,
 -0.0479736328125,
 0.12158203125]

In [22]:
# Create ideal results
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import exact_observables

ideal_expvals = exact_observables(circuit, observables)
ideal_expvals

array([0.50390696, 0.57454157, 0.39584538, 0.09798816, 0.18481229,
       0.23530298])

In [23]:
# Compare the error between results
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import compare_results

compare_results(quantinuum_expvals, ideal_expvals)

Simulated expectation values: [0.32055664, 0.5144043, 0.34899902, 0.13574219, -0.04797363, 0.12158203]
Exact expectation values: [0.50390696, 0.57454157, 0.39584538, 0.09798816, 0.18481229, 0.23530298]
Errors in estimation: [-0.18335032, -0.06013727, -0.04684636, 0.03775402, -0.23278593, -0.11372094]
Relative errors in estimation: [-0.36385749, -0.10467001, -0.1183451, 0.38529168, -1.25958031, -0.48329582]


The below is a cost estimator for the circuits run - make sure to add up for all subcircuits run.

In [24]:
cost_backend = provider.get_backend('quantinuum.qpu.h1-1')
total_cost = 0
for sample in subexperiments:
    for partition in sample:
        for subexperiment in partition:
            cost = cost_backend.estimate_cost(subexperiment, shots=128)
            total_cost += cost.estimated_total
print(f'Estimated cost: {total_cost} {cost.currency_code}')

Estimated cost: 133.8752 HQC
